In [ ]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Model

In [6]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model_CTC = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and

In [11]:
librispeech_samples_ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# load audio
audio_input, sample_rate = sf.read(librispeech_samples_ds[1]["file"])

# pad input values and return pt tensor
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

# INFERENCE

hidden_states = model(input_values).last_hidden_state
# retrieve logits & take argmax
logits = model_CTC(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe
transcription = processor.decode(predicted_ids[0])
print(input_values.shape)
print(hidden_states.shape)
print(logits.shape)
print(transcription)

Reusing dataset librispeech_asr (/home/slzhang/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/468ec03677f46a8714ac6b5b64dba02d246a228d92cbbad7f3dc190fa039eab1)


torch.Size([1, 77040])
torch.Size([1, 240, 768])
torch.Size([1, 240, 32])
NOR IS MISTER QUILTER'S MANNER LESS INTERESTING THAN HIS MATTER


In [12]:
model.config

Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_dropout_prob": 0.1,
  "hidden_size":

In [10]:
dummy_input = torch.randn(1, 200000)
torch.onnx.export(model, dummy_input, "./Wav2Vec_model.onnx")

/home/slzhang/miniconda/envs/fedml/lib/python3.7/site-packages/torch/nn/functional.py:2110: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if size_prods == 1:
/home/slzhang/miniconda/envs/fedml/lib/python3.7/site-packages/transformers-4.8.1-py3.8.egg/transformers/models/wav2vec2/modeling_wav2vec2.py:438: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/home/slzhang/miniconda/envs/fedml/lib/python3.7/site-packages/transformers-4.8.1-py3.8.egg/transformers/models/wav2vec2/modeling_wav2vec2.py:475: TracerWarning: Converting a tensor to a Python boolean might c

In [8]:
# FINE-TUNE

target_transcription = "A MAN SAID TO THE UNIVERSE"

# encode labels
with processor.as_target_processor():
  labels = processor(target_transcription, return_tensors="pt").input_ids

In [9]:
extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")
decoder = Wav2Vec2CTCTokenizer
input_values = extractor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = decoder.decode(predicted_ids[0])
transcription


AttributeError: 'Wav2Vec2BaseModelOutput' object has no attribute 'logits'

In [62]:
for k,v in model.named_parameters():
    print(k)

wav2vec2.masked_spec_embed
wav2vec2.feature_extractor.conv_layers.0.conv.weight
wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight
wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias
wav2vec2.feature_extractor.conv_layers.1.conv.weight
wav2vec2.feature_extractor.conv_layers.2.conv.weight
wav2vec2.feature_extractor.conv_layers.3.conv.weight
wav2vec2.feature_extractor.conv_layers.4.conv.weight
wav2vec2.feature_extractor.conv_layers.5.conv.weight
wav2vec2.feature_extractor.conv_layers.6.conv.weight
wav2vec2.feature_projection.layer_norm.weight
wav2vec2.feature_projection.layer_norm.bias
wav2vec2.feature_projection.projection.weight
wav2vec2.feature_projection.projection.bias
wav2vec2.encoder.pos_conv_embed.conv.bias
wav2vec2.encoder.pos_conv_embed.conv.weight_g
wav2vec2.encoder.pos_conv_embed.conv.weight_v
wav2vec2.encoder.layer_norm.weight
wav2vec2.encoder.layer_norm.bias
wav2vec2.encoder.layers.0.attention.k_proj.weight
wav2vec2.encoder.layers.0.attention.k_proj.bias
wav2ve

In [7]:
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
from transformers.modeling_outputs import CausalLMOutput
from wav2vec2_model import Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Model, Wav2Vec2PreTrainedModel, Wav2Vec2Config, Wav2Vec2Encoder, Wav2Vec2EncoderLayer
import torch.nn as nn
import copy

class Wav2Vec2_with_exit(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.configuration_s1 = copy.deepcopy(config)
        self.configuration_s2 = copy.deepcopy(config)

        self.wav2vec2 = Wav2Vec2Model(config)
        self.dropout = nn.Dropout(config.final_dropout)

        if config.vocab_size is None:
            raise ValueError(
                f"You are trying to instantiate {self.__class__} with a configuration that "
                "does not define the vocabulary size of the language model head. Please "
                "instantiate the model as follows: `Wav2Vec2ForCTC.from_pretrained(..., vocab_size=vocab_size)`."
                "or define `vocab_size` of your model's configuration."
            )
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)

        self.init_weights()

    def add_exit(self, placement):

        self.configuration_s1.num_hidden_layers = placement
        self.configuration_s2.num_hidden_layers -= placement
        self.wav2vec2_s1 = Wav2Vec2Model(self.configuration_s1)
        self.wav2vec2_s2 = nn.ModuleList([Wav2Vec2EncoderLayer(self.configuration_s2) for _ in range(self.configuration_s2.num_hidden_layers)])

        dict_pre_trained = self.wav2vec2.state_dict().copy()
        dict_s1 = self.wav2vec2_s1.state_dict().copy()
        dict_s2 = self.wav2vec2_s2.state_dict().copy()
        for k,v in self.wav2vec2.state_dict().items():

            for k_s1,v_s1 in self.wav2vec2_s1.state_dict().items():
                if k.split(".")[-4:] == k_s1.split(".")[-4:]:
                    dict_s1[k_s1] = dict_pre_trained[k]

            for k_s2,v_v2 in self.wav2vec2_s2.state_dict().items():
                if k.split(".")[-3:] == k_s2.split(".")[-3:] and int(k.split(".")[2]) == int(k_s2.split(".")[0])+placement:
                    dict_s2[k_s2] = dict_pre_trained[k]

        self.wav2vec2_s1.load_state_dict(dict_s1)
        self.wav2vec2_s2.load_state_dict(dict_s2)

    def freeze_feature_extractor(self):
        """
        Calling this function will disable the gradient computation for the feature extractor so that its parameter
        will not be updated during training.
        """
        self.wav2vec2_s1.feature_extractor._freeze_parameters()

    def forward(
        self,
        input_values,
        attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        s1_outputs = self.wav2vec2_s1(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        s1_hidden_states = s1_outputs[0]
        s1_hidden_states = self.dropout(s1_hidden_states)
        s1_logits = self.lm_head_exit(s1_hidden_states)

        s2_outputs = self.wav2vec2_s2(
            s1_hidden_states, 
            attention_mask=attention_mask, 
            output_attentions=output_attentions
        )

        s2_hidden_states = s2_outputs[0]
        s2_hidden_states = self.dropout(s2_hidden_states)
        s2_logits = self.lm_head(s2_hidden_states)

        loss = None
        if labels is not None:

            if labels.max() >= self.config.vocab_size:
                raise ValueError(f"Label values must be <= vocab_size: {self.config.vocab_size}")

            # retrieve loss input_lengths from attention_mask
            attention_mask = (
                attention_mask if attention_mask is not None else torch.ones_like(input_values, dtype=torch.long)
            )
            input_lengths = self._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)

            # assuming that padded tokens are filled with -100
            # when not being attended to
            labels_mask = labels >= 0
            target_lengths = labels_mask.sum(-1)
            flattened_targets = labels.masked_select(labels_mask)

            # ctc_loss doesn't support fp16
            s1_log_probs = nn.functional.log_softmax(s1_logits, dim=-1, dtype=torch.float32).transpose(0, 1)
            s2_log_probs = nn.functional.log_softmax(s2_logits, dim=-1, dtype=torch.float32).transpose(0, 1)

            with torch.backends.cudnn.flags(enabled=False):
                s1_loss = nn.functional.ctc_loss(
                    s1_log_probs,
                    flattened_targets,
                    input_lengths,
                    target_lengths,
                    blank=self.config.pad_token_id,
                    reduction=self.config.ctc_loss_reduction,
                    zero_infinity=self.config.ctc_zero_infinity,
                )
                s2_loss = nn.functional.ctc_loss(
                    s2_log_probs,
                    flattened_targets,
                    input_lengths,
                    target_lengths,
                    blank=self.config.pad_token_id,
                    reduction=self.config.ctc_loss_reduction,
                    zero_infinity=self.config.ctc_zero_infinity,
                )
                loss = 0.5 * s1_loss + 0.5 * s2_loss

        if not return_dict:
            output = (s2_logits,) + s2_outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return CausalLMOutput(
            loss=loss, logits=s2_logits, hidden_states=s2_outputs.hidden_states, attentions=s2_outputs.attentions
        )

In [8]:
model_CTC = Wav2Vec2_with_exit.from_pretrained("facebook/wav2vec2-base-960h")
model_CTC.add_exit(5)
model_CTC.freeze_feature_extractor()

Some weights of Wav2Vec2_with_exit were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


masked_spec_embed
feature_extractor.conv_layers.0.conv.weight
feature_extractor.conv_layers.0.layer_norm.weight
feature_extractor.conv_layers.0.layer_norm.bias
feature_extractor.conv_layers.1.conv.weight
feature_extractor.conv_layers.2.conv.weight
feature_extractor.conv_layers.3.conv.weight
feature_extractor.conv_layers.4.conv.weight
feature_extractor.conv_layers.5.conv.weight
feature_extractor.conv_layers.6.conv.weight
feature_projection.layer_norm.weight
feature_projection.layer_norm.bias
feature_projection.projection.weight
feature_projection.projection.bias
encoder.pos_conv_embed.conv.bias
encoder.pos_conv_embed.conv.weight_g
encoder.pos_conv_embed.conv.weight_v
encoder.layer_norm.weight
encoder.layer_norm.bias
encoder.layers.0.attention.k_proj.weight
encoder.layers.0.attention.k_proj.bias
encoder.layers.0.attention.v_proj.weight
encoder.layers.0.attention.v_proj.bias
encoder.layers.0.attention.q_proj.weight
encoder.layers.0.attention.q_proj.bias
encoder.layers.0.attention.out_proj

In [15]:
for k,v in model_CTC.named_parameters():
    print(k)

wav2vec2_s1.masked_spec_embed
wav2vec2_s1.feature_extractor.conv_layers.0.conv.weight
wav2vec2_s1.feature_extractor.conv_layers.0.layer_norm.weight
wav2vec2_s1.feature_extractor.conv_layers.0.layer_norm.bias
wav2vec2_s1.feature_extractor.conv_layers.1.conv.weight
wav2vec2_s1.feature_extractor.conv_layers.2.conv.weight
wav2vec2_s1.feature_extractor.conv_layers.3.conv.weight
wav2vec2_s1.feature_extractor.conv_layers.4.conv.weight
wav2vec2_s1.feature_extractor.conv_layers.5.conv.weight
wav2vec2_s1.feature_extractor.conv_layers.6.conv.weight
wav2vec2_s1.feature_projection.layer_norm.weight
wav2vec2_s1.feature_projection.layer_norm.bias
wav2vec2_s1.feature_projection.projection.weight
wav2vec2_s1.feature_projection.projection.bias
wav2vec2_s1.encoder.pos_conv_embed.conv.bias
wav2vec2_s1.encoder.pos_conv_embed.conv.weight_g
wav2vec2_s1.encoder.pos_conv_embed.conv.weight_v
wav2vec2_s1.encoder.layer_norm.weight
wav2vec2_s1.encoder.layer_norm.bias
wav2vec2_s1.encoder.layers.0.attention.k_proj.w

In [27]:
print(model_CTC.wav2vec2.modules())

<generator object Module.modules at 0x7f2678576b50>
